In [1]:
from sx_multi.data_source import DataSource
from sx_multi.analysis import Analysis
from sx_multi.local_executor import LocalExecutor

from sx_multi.dask_executor import DaskExecutor

import matplotlib.pyplot as plt



from servicex import ServiceXDataset
from sx_multi import FuncAdlQastle
from coffea import hist, processor


In [2]:
dids = ['mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00']
datasets = [
    ServiceXDataset(did, backend_type='xaod')
    for did in dids
]

In [3]:
ds = FuncAdlQastle()
leptons_per_event_query = ds \
        .Select(lambda e: e.Electrons("Electrons")) \
        .Select(lambda eles: eles.Where(lambda e: e.pt()/1000.0 > 30.0)) \
        .Select(lambda eles: eles.Where(lambda e: abs(e.eta()) < 2.5)) \
        .Where(lambda eles: len(eles) == 2) \
        .Select(lambda ls: (ls.Select(lambda e: e.pt()/1000.0), ls.Select(lambda e: e.eta()), ls.Select(lambda e: e.phi()), ls.Select(lambda e: e.m()/1000.0))) \
        .AsROOTTTree('data.root', 'mytree', ('ElePt', 'EleEta', 'ElePhi', 'EleM'))

leptons_per_event_query

In [4]:
datasource = DataSource(query=leptons_per_event_query, metadata={}, datasets=datasets)

In [5]:
class Z_EEAnalysis(Analysis):
    def __init__(self):
        self.accumulator = processor.dict_accumulator({
            "sumw": processor.defaultdict_accumulator(float),
            "mass": hist.Hist(
                "Events",
                hist.Cat("dataset", "Dataset"),
                hist.Bin("mass", "$Z_{ee}$ [GeV]", 60, 60, 120),
            ),
        })

    @staticmethod
    def process(accumulator, events=None):
        import awkward1 as ak
        dataset = events.metadata['dataset']
        electrons = ak.zip({
            "pt": events.ElePt,
            "eta": events.EleEta,
            "phi": events.ElePhi,
            "mass": events.EleM,
            "charge": events.EleM,
        }, with_name="PtEtaPhiMCandidate")

        # The template of the output we will send back.
        output = accumulator.identity()

        # Do the buts, form the invar mass, plot.
        cut = (ak.num(electrons) == 2)
        diele = electrons[cut][:, 0] + electrons[cut][:, 1]

        output["sumw"][dataset] += len(events)
        output["mass"].fill(
            dataset=dataset,
            mass=diele.mass,
        )

        return output



In [8]:
analysis = Z_EEAnalysis()
executor = DaskExecutor(client_addr="127.0.0.1:8080")


None


/Users/bengal1/.local/share/virtualenvs/funcx_coffea/lib/python3.7/site-packages/distributed/client.py:1129: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | client | scheduler | workers |
+-------------+--------+-----------+---------+
| blosc       | None   | 1.9.2     | None    |
| distributed | 2.30.1 | 2.30.0    | None    |
| tornado     | 6.1    | 6.0.4     | None    |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [9]:
%matplotlib widget
async def plot_stream(accumulator_stream):
  async for coffea_info in accumulator_stream:
    # Need to ask coffea folks how to anomate this!
    hist.plot1d(coffea_info['mass'])
    plt.show()
  return coffea_info

await plot_stream(executor.execute(analysis, datasource))

{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 24000.0}), 'mass': <Hist (dataset,mass) instance at 0x132d5cc10>}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 45000.0}), 'mass': <Hist (dataset,mass) instance at 0x132e2f190>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 50000.0}), 'mass': <Hist (dataset,mass) instance at 0x132e5f250>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 50000.0}), 'mass': <Hist (dataset,mass) instance at 0x132e8af10>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 50000.0}), 'mass': <Hist (dataset,mass) instance at 0x132e8aed0>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 125000.0}), 'mass': <Hist (dataset,mass) instance at 0x132f118d0>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0}), 'mass': <Hist (dataset,mass) instance at 0x132daf1d0>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0}), 'mass': <Hist (dataset,mass) instance at 0x132be0c50>}
{'sumw': defaultdict_accumulator(<class 'float'>, {'mc15x': 150000.0}), 'mass': <Hist (dataset,mass) instance at 0x

{'sumw': defaultdict_accumulator(float, {'mc15x': 1993800.0}),
 'mass': <Hist (dataset,mass) instance at 0x131ae54d0>}